In [ ]:
from google.colab import files
uploaded = files.upload()

Saving Analyst_assignment.xlsx to Analyst_assignment.xlsx


In [ ]:
import pandas as pd

# File path and sheet name
file_path = "Analyst_assignment.xlsx"
sheet_name = "დავალება 3"

# Read the Excel file into a pandas DataFrame with the second row as column names
df = pd.read_excel(file_path, sheet_name=sheet_name, header=1)

# Print the column names
print(df.columns)


Index(['Cif - Customer ID', 'Is Feature Person', 'Operation Type',
       'User ID-Tanamshromlis unikaluri kodi', 'Region', 'Branch',
       'Operation Date', 'source'],
      dtype='object')


In [ ]:
import pandas as pd

# File path and sheet name
file_path = "Analyst_assignment.xlsx"
sheet_name = "დავალება 3"

# Read the Excel file into a pandas DataFrame with the correct column names
df = pd.read_excel(file_path, sheet_name=sheet_name, header=1)

# Column names
branch_col = "Branch"
operation_date_col = "Operation Date"

# Calculate the total workload for each branch
branch_workload = df.groupby(branch_col)[operation_date_col].count().reset_index()
branch_workload.rename(columns={operation_date_col: "TotalWorkload"}, inplace=True)

# Total number of employees
total_employees = 81

# Calculate the proportional allocation factor based on total workload
allocation_factor = total_employees / branch_workload["TotalWorkload"].sum()

# Calculate the number of employees needed per branch
branch_workload["EmployeesNeeded"] = (branch_workload["TotalWorkload"] * allocation_factor).round().astype(int)

# Calculate the number of employees currently assigned to each branch
branch_employees = df.groupby(branch_col)["User ID-Tanamshromlis unikaluri kodi"].nunique().reset_index()
branch_employees.rename(columns={"User ID-Tanamshromlis unikaluri kodi": "EmployeesAssigned"}, inplace=True)

# Calculate the number of employees to redistribute
branch_workload = pd.merge(branch_workload, branch_employees, on=branch_col, how="left")
branch_workload["EmployeesRedistributed"] = branch_workload["EmployeesNeeded"] - branch_workload["EmployeesAssigned"]

# Distribute employees from branches with excess to branches in need
excess_branches = branch_workload[branch_workload["EmployeesRedistributed"] > 0]
need_branches = branch_workload[branch_workload["EmployeesRedistributed"] < 0]

while len(excess_branches) > 0 and len(need_branches) > 0:
    excess_branches = excess_branches.sort_values("EmployeesRedistributed", ascending=False)
    need_branches = need_branches.sort_values("EmployeesRedistributed")
    
    excess_index = excess_branches.index[0]
    need_index = need_branches.index[0]
    
    excess = excess_branches.loc[excess_index, "EmployeesRedistributed"]
    need = need_branches.loc[need_index, "EmployeesRedistributed"]
    
    if excess >= -need:
        transfer = -need
        excess_branches.loc[excess_index, "EmployeesRedistributed"] -= transfer
        need_branches.loc[need_index, "EmployeesRedistributed"] -= transfer
        branch_workload.loc[need_index, "EmployeesAssigned"] -= transfer
        branch_workload.loc[excess_index, "EmployeesAssigned"] += transfer
        excess_branches = excess_branches[excess_branches["EmployeesRedistributed"] > 0]
        need_branches = need_branches[need_branches["EmployeesRedistributed"] < 0]
    else:
        break

# Display the optimized distribution
print(branch_workload)


   Branch  TotalWorkload  EmployeesNeeded  EmployeesAssigned  \
0       a           3049                6                 12   
1       b           2734                6                  9   
2       c           1822                4                  4   
3       d           3311                7                  8   
4       e           3689                8                  5   
5       f           2808                6                 10   
6       g           3243                7                 11   
7       h            177                0                  3   
8       i           3795                8                 10   
9       j           1902                4                  4   
10      k           1838                4                  5   
11      o           2805                6                  4   
12      p           3229                7                  6   
13      q            777                2                  4   
14      r           1234                

In [ ]:
# Input data
data = {
    "EmployeesRedistributed": [6, 3, 0, 1, -3, 4, 4, 3, 2, 0, 1, -2, -1, 2, 1, 1, -1]
}

# Create a DataFrame from the input data
df = pd.DataFrame(data)

# Adjust the values by subtracting from positive numbers
for i in range(len(df)):
    if df.at[i, "EmployeesRedistributed"] < 0:
        value = abs(df.at[i, "EmployeesRedistributed"])
        for j in range(len(df)):
            if df.at[j, "EmployeesRedistributed"] > 0:
                if df.at[j, "EmployeesRedistributed"] >= value:
                    df.at[j, "EmployeesRedistributed"] -= value
                    value = 0
                    break
                else:
                    value -= df.at[j, "EmployeesRedistributed"]
                    df.at[j, "EmployeesRedistributed"] = 0
    df.at[i, "EmployeesRedistributed"] = max(0, df.at[i, "EmployeesRedistributed"])

# Print the result
print(df["EmployeesRedistributed"])


0     0
1     2
2     0
3     1
4     0
5     4
6     4
7     3
8     2
9     0
10    1
11    0
12    0
13    2
14    1
15    1
16    0
Name: EmployeesRedistributed, dtype: int64


In [ ]:
# Input data
data = {
    "EmployeesRedistributed": [0, 2, 0, 1, 0, 4, 4, 3, 2, 0, 1, 0, 0, 2, 1, 1, 0]
}

# Create a DataFrame from the input data
df = pd.DataFrame(data)

# Adjust the values by redistributing them
zeros_indices = df.index[df["EmployeesRedistributed"] == 0]
positive_indices = df.index[df["EmployeesRedistributed"] > 0]

for zero_index in zeros_indices:
    while df.at[zero_index, "EmployeesRedistributed"] < 1:
        for pos_index in positive_indices:
            if df.at[pos_index, "EmployeesRedistributed"] > 1:
                df.at[zero_index, "EmployeesRedistributed"] += 1
                df.at[pos_index, "EmployeesRedistributed"] -= 1
            if df.at[pos_index, "EmployeesRedistributed"] == 1:
                positive_indices = positive_indices.drop(pos_index)
            if df.at[zero_index, "EmployeesRedistributed"] >= 1:
                break

# Print the result
print(df["EmployeesRedistributed"])


0     1
1     1
2     1
3     1
4     1
5     1
6     1
7     3
8     2
9     1
10    1
11    1
12    1
13    2
14    1
15    1
16    1
Name: EmployeesRedistributed, dtype: int64
